# DeepSeek-V3.1 Model Getting Started Guide on Amazon Bedrock

This notebook provides a comprehensive introduction to using DeepSeek-V3.1 on Amazon Bedrock, including how to leverage the familiar OpenAI SDK interface with Amazon Bedrock. We'll cover how to make API requests, explore available parameters and payload structures, and examine use cases for this advanced reasoning model. 

## Model Overview

### DeepSeek-V3.1

**Parameters:** 685 billion (Mixture-of-Experts hybrid model)

**Use Cases:** Complex reasoning tasks, agentic use cases, thinking and non-thinking modes

**Key Features:**
- **Thinking Mode**: Carefully works through problems step-by-step with enhanced reasoning
- **Non-Thinking Mode**: Provides quick responses to straightforward questions
- **Hybrid Architecture**: Mixture-of-Experts (MoE) design for optimal performance
- **Enhanced Tool Calling**: Superior performance in agent-based tasks

## Core Capabilities

DeepSeek-V3.1 offers the following characteristics:

**Input/Output:** Text-in, text-out 

**Context Window:** 128,000 tokens  

**Model Type:** Advanced reasoning model with thinking capabilities

**Languages:** English and Chinese

**Tool Calling:** ✅ Supported (Enhanced capabilities)

**Bedrock Guardrails** ✅ Supported

**Converse API** ✅ Supported

**OpenAI Chat Completions API** ✅ Supported

**Streaming:** ✅ Supported

**Model Evaluation:** ✅ Supported

**Agents:** ✅ Supported

**Prompt Management:** ✅ Supported

**Flows:** ✅ Supported

**Batch Inference:** ✅ Supported

## What You'll Learn in this getting started guide

- Options to use Amazon Bedrock for DeepSeek-V3.1 inference, including:    
    - Using the OpenAI SDK with Amazon Bedrock
    - Using Amazon Bedrock's InvokeModel API
    - Using Amazon Bedrock's Converse API
- Understanding request parameters and response structures
- Leveraging thinking vs non-thinking modes for different use cases
- Implementing enhanced tool calling capabilities
- Exploring reasoning capabilities with thinking mode


## Model Access on Amazon Bedrock

Ensure you have the correct IAM permission in order to access DeepSeek's models on Amazon Bedrock. 

## IAM Permissions

To use Bedrock models, your AWS credentials need the following permissions:


```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:InvokeModel",
        "bedrock:InvokeModelWithResponseStream"
      ],
      "Resource": "arn:aws:bedrock:*::foundation-model/deepseek.v3-v1:0"
    }
  ]
}
```

**Note:** The wildcard (`*`) in the region field covers all supported regions

## Step 1: Environment Configuration

First, we need to install the required packages and tell the OpenAI SDK to talk to Bedrock instead of OpenAI's servers.

### Required Imports:
- `os` → For environment variables
- `boto3` → For native Bedrock API interactions  
- `json` → For JSON serialization/deserialization
- `datetime` → For timestamp tracking and performance measurements
- `openai` → For OpenAI SDK compatibility with Bedrock
- `IPython.display` → For enhanced output formatting and streaming demonstrations

### Environment Variables:
We set two environment variables to redirect the OpenAI SDK:
- `AWS_BEARER_TOKEN_BEDROCK` → Your Bedrock API key  
- `OPENAI_BASE_URL` → Bedrock's OpenAI-compatible endpoint


In [ ]:
!pip install boto3 openai ipython


In [ ]:
import os
import boto3
import json
from openai import OpenAI
from datetime import datetime
from IPython.display import clear_output, display, display_markdown, Markdown


### Model ID

- **deepseek.v3-v1:0**


In [ ]:
# Model Configuration - DeepSeek-V3.1
MODEL_ID = "deepseek.v3-v1:0"  

print(f"✅ Using model: {MODEL_ID}")


In [ ]:
# Set environment variables to point to Bedrock
# Note: Change the region in the URL to match your preferred region
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = "bedrock-api-key"
os.environ["OPENAI_API_KEY"] = "bedrock-api-key"
os.environ["OPENAI_BASE_URL"] = "https://bedrock-runtime.us-west-2.amazonaws.com/openai/v1"

print("✅ Environment configured for Bedrock!")
print("📍 Using us-west-2 region - change the URL above to use a different region")


## Step 2: Inference with Amazon Bedrock

### Option 1: OpenAI SDK

#### Import and Initialize OpenAI Client

Now we use the **exact same OpenAI SDK** you're familiar with. The client will automatically read the environment variables we just set.

**Key Point**: This is the same OpenAI library, but now it's talking to Amazon Bedrock.


In [ ]:
# Initialize both clients
# Note: Change region_name to match your preferred region
client = OpenAI()  # For chat completions API
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')  

print("✅ OpenAI client initialized (pointing to Bedrock)")
print(f"✅ Bedrock client initialized in region: {bedrock_client.meta.region_name}")
print("📍 Change region_name above to use a different supported region")


#### Make API Calls 

The API call structure is identical to OpenAI:
- Same `messages` format with `role` and `content`
- Same `model` parameter (but uses Bedrock model IDs)  
- Same `stream` parameter for real-time responses
- **New**: `thinking_mode` parameter to control thinking vs non-thinking behavior


In [ ]:
# Example 1: Non-thinking mode (quick response)
response = client.chat.completions.create(
    model=MODEL_ID,                 
    messages=[
        {"role": "system", "content": "You are a concise, highly logical assistant."},
        {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
    ],
    temperature=0,
    max_completion_tokens=1000,
    # Non-thinking mode for quick responses

)

# Extract and print the response text
print("🤖 Non-thinking mode response:")
print(response.choices[0].message.content)


In [ ]:
# Example 2: Thinking mode (step-by-step reasoning)
response = client.chat.completions.create(
    model=MODEL_ID,                 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that thinks through problems step by step."},
        {"role": "user",   "content": "If a train leaves station A at 60 mph and another leaves station B at 40 mph, and they are 200 miles apart, when will they meet?"}
    ],
    temperature=0,
    max_completion_tokens=2000,
    # reasoning_effort values: low, medium, high
    reasoning_effort='high'  # Thinking mode for complex reasoning
)

# Extract and print the response text
print("🧠 Thinking mode response:")
print(response.choices[0].message.content)


#### Process Streaming Response

Handle the response exactly like you would with OpenAI's SDK. Each `item` in the response is a chunk of the model's output. DeepSeek-V3.1 supports streaming in both thinking and non-thinking modes.

In [ ]:
# Streaming with thinking mode
streaming_response = client.chat.completions.create(
    model=MODEL_ID,                 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that thinks through problems step by step."},
        {"role": "user",   "content": "Explain how photosynthesis works in simple terms."}
    ],
    temperature=0,
    max_completion_tokens=1500,
    reasoning_effort='high',  # Enable thinking mode
    stream=True
)

# Extract and print the response text in real-time.
print("🧠 Streaming thinking mode response:")
for chunk in streaming_response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


#### What's Happening Behind the Scenes?

When you use the OpenAI SDK with Bedrock, your requests are automatically translated to Bedrock's native `InvokeModel` API.

#### Request Translation
- **OpenAI SDK Request** → **Bedrock InvokeModel** 
- The request body structure remains the same
- But there are some key differences in how parameters are handled:

| Parameter | OpenAI SDK | Bedrock InvokeModel |
|-----------|------------|-------------------|
| **Model ID** | In request body | Part of the URL path |
| **Streaming** | `stream=True/False` | Different API endpoints:<br/>• `InvokeModel` (non-streaming)<br/>• `InvokeModelWithResponseStream` (streaming) |
| **Request Body** | Full chat completions format | Same format, but `model` and `stream` are optional |
| **Thinking Mode** | `thinking_mode=True/False` | DeepSeek-V3.1 specific parameter |


#### Enhanced Function Calling with OpenAI SDK

DeepSeek-V3.1 features enhanced tool calling capabilities for superior performance in agent-based tasks. Let's demonstrate this with a weather lookup function.

In [ ]:
def get_weather(location):
    """
    Get current weather for a given location.
    This is a mock function that returns sample weather data for demonstration.
    
    Args:
        location (str): City name or "City, Country" format
        
    Returns:
        dict: Weather information with temperature, condition, and humidity
    """
    # Enhanced mock weather data - in a real application, you'd call a weather API
    weather_data = {
        # Exact matches
        "Paris, France": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "Tokyo, Japan": {"temperature": "25°C", "condition": "Light rain", "humidity": "80%"},
        "New York, USA": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Sydney, Australia": {"temperature": "28°C", "condition": "Clear skies", "humidity": "55%"},
        "London, UK": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"},
        
        # City-only matches for more flexible parsing
        "Paris": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "Tokyo": {"temperature": "25°C", "condition": "Light rain", "humidity": "80%"},
        "New York": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Sydney": {"temperature": "28°C", "condition": "Clear skies", "humidity": "55%"},
        "London": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"}
    }
    
    return weather_data.get(location, {
        "temperature": "23°C", 
        "condition": "Pleasant", 
        "humidity": "60%"
    })

# Define the function schema for OpenAI SDK
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature and weather conditions for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country, e.g. 'Paris, France'"
                }
            },
            "required": ["location"],
            "additionalProperties": False
        }
    }
}]

print("✅ Weather function and tools configuration ready!")


In [ ]:
def chat_with_functions(client, model, messages, tools, max_iterations=3, reasoning_effort='high'):
    """
    Chat with function calling support using OpenAI SDK format.
    
    Args:
        client: OpenAI client instance
        model: Model ID to use
        messages: List of conversation messages
        tools: List of available tools/functions
        max_iterations: Maximum number of function call iterations
        reasoning_effort: Whether to use thinking mode for enhanced reasoning
        
    Returns:
        Final assistant message
    """
    
    for iteration in range(max_iterations):
        print(f"🔄 Iteration {iteration + 1}")
        
        # Make request with tools
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            reasoning_effort=reasoning_effort  # Use thinking mode for enhanced tool calling
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Check if the model wants to call functions
        if assistant_message.tool_calls:
            print(f"🔧 Model requested {len(assistant_message.tool_calls)} function call(s)")
            
            # Process each function call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"🔧 Calling function: {function_name}")
                print(f"🔧 Arguments: {function_args}")
                
                # Call the actual function
                if function_name == "get_weather":
                    function_result = get_weather(function_args["location"])
                    print(f"🔧 Function result: {function_result}")
                else:
                    function_result = {"error": f"Unknown function: {function_name}"}
                
                # Add function result to conversation
                function_message = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": json.dumps(function_result)
                }
                messages.append(function_message)
                
        else:
            # No more function calls, return final response
            print("✅ No function calls requested, conversation complete")
            return assistant_message
    
    print("⚠️ Maximum iterations reached")
    return assistant_message

print("✅ Enhanced function calling handler ready!")


In [ ]:
# Test enhanced function calling with thinking mode
weather_questions = [
    "What's the weather like in Paris today?",
    "Can you tell me the temperature in Tokyo?",
    "How's the weather in Sydney, Australia?",
    "What are the conditions like in New York?"
]

print("🌤️ Testing Enhanced Function Calling with DeepSeek-V3.1")
print("=" * 60)

for i, question in enumerate(weather_questions, 1):
    print(f"\n📝 Test {i}: {question}")
    print("-" * 40)
    
    try:
        # Create conversation messages
        messages = [
            {"role": "system", "content": "You are a helpful weather assistant. When users ask about weather, temperature, or weather conditions for any location, you MUST call the get_weather function to get current information. Always use the function when weather information is requested."},
            {"role": "user", "content": question}
        ]
        
        # Test function calling directly with tool_choice="required"
        print("🔧 Testing function call...")
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=messages,
            tools=tools,
            tool_choice="required",  # Force function calling
            reasoning_effort='high'
        )
        
        if response.choices[0].message.tool_calls:
            tool_call = response.choices[0].message.tool_calls[0]
            print(f"✅ Function called successfully: {tool_call.function.name}")
            print(f"🔧 Arguments: {tool_call.function.arguments}")
            
            # Execute the function and show result
            import json
            args = json.loads(tool_call.function.arguments)
            function_result = get_weather(args.get("location", "Unknown"))
            print(f"🔧 Function result: {function_result}")
            
            # Generate a dummy response based on the function result
            location = args.get("location", "Unknown")
            temp = function_result.get("temperature", "Unknown")
            condition = function_result.get("condition", "Unknown")
            humidity = function_result.get("humidity", "Unknown")
            
            dummy_response = f"Based on current weather data for {location}: Temperature is {temp} with {condition.lower()} conditions and {humidity} humidity. Great for planning your day!"
            
            print("🤖 Generated weather report:")
            print(dummy_response)
            print("✅ Function calling demonstration complete!")
            
        else:
            print("❌ No function call made")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print()


#### What Just Happened with Enhanced Function Calling?

The enhanced function calling demonstration shows DeepSeek-V3.1's improved capabilities:

1. **Enhanced Tool Recognition**: DeepSeek-V3.1 has superior tool calling performance for agent-based tasks
2. **Thinking Mode Integration**: When `thinking_mode=True`, the model can reason through complex tool selection and usage
3. **Improved Function Execution**: Better understanding of when and how to use available tools
4. **Multi-step Reasoning**: The model can plan and execute complex multi-tool workflows
5. **Context Awareness**: Enhanced understanding of conversation context for better tool usage decisions

**Key Advantages of DeepSeek-V3.1's Enhanced Tool Calling:**
- More accurate tool selection based on user intent
- Better handling of complex multi-step agent workflows  
- Improved reasoning about tool parameters and results
- Enhanced error handling and recovery in tool usage scenarios


### Option 2: Amazon Bedrock's InvokeModel API

The Bedrock InvokeModel API is the foundational interface for interacting directly with any model hosted on Amazon Bedrock. It provides low-level, flexible access to model inference, allowing you to send input data and receive generated responses in a consistent way across all supported models.

**Key Benefits:**
- Direct Access: Interact with any Bedrock model using a unified API endpoint.
- Fine-Grained Control: Customize inference parameters and payloads for each request.
- Streaming Support: Use `InvokeModelWithResponseStream` for real-time, token-by-token output.
- Privacy: Amazon Bedrock does not store your input or output data—requests are used only for inference.
- **Thinking Mode Control**: Direct control over DeepSeek-V3.1's thinking vs non-thinking behavior.


#### Setup client

First, we setup the Amazon Bedrock client.

In [ ]:
# Configure region for Bedrock client
region = "us-west-2"

if region is None:
    target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
else:
    target_region = "us-west-2"

bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)
print(f"📍 Using region: {target_region} - change the region variable above to use a different supported region")


#### Inference with InvokeModel API

Then we use the InvokeModel API to perform model inference with DeepSeek-V3.1.

In [ ]:
def invoke_model(body, model_id, accept, content_type):
    """
    Invokes Amazon bedrock model to run an inference
    using the input provided in the request body.
    
    Args:
        body (dict): The invokation body to send to bedrock
        model_id (str): the model to query
        accept (str): input accept type
        content_type (str): content type
    Returns:
        Inference response from the model.
    """

    try:
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body), 
            modelId=model_id, 
            accept=accept, 
            contentType=content_type
        )

        return response

    except Exception as e:
        print(f"Couldn't invoke {model_id}")
        raise e


In [ ]:
# Example with thinking mode enabled
messages = [
    {"role": "system", "content": "You are a concise, highly logical assistant."},
    {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
]

body = {
    "messages": messages,
    "temperature": 0,
    "max_completion_tokens": 1000,
    "reasoning_effort": "high"  # Enable thinking mode for DeepSeek-V3.1
}

accept = "application/json"
contentType = "application/json"

response = invoke_model(body, MODEL_ID, accept, contentType)
response_body = json.loads(response.get("body").read())

print("🧠 Thinking mode response:")
print(response_body['choices'][0]['message']['content'])


#### Streaming with InvokeModel API

The InvokeModel API comes with built in streaming support. This can be useful in user-facing applications since it reduces time to first token (TTFT) metric and with that perceived inference latency for the end user. DeepSeek-V3.1 supports streaming in both thinking and non-thinking modes.

In [ ]:
# Streaming with thinking mode
messages = [
    {"role": "system", "content": "You are a concise, highly logical assistant."},
    {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
]

body = {
    "messages": messages,
    "temperature": 0,
    "max_completion_tokens": 1000,
    "reasoning_effort": "high"  # Enable thinking mode for streaming
}

accept = "application/json"
contentType = "application/json"

start_time = datetime.now()

response = bedrock_runtime.invoke_model_with_response_stream(
    body=json.dumps(body), modelId=MODEL_ID, accept=accept, contentType=contentType
)
chunk_count = 0
time_to_first_token = None

# Process the response stream
stream = response.get("body")
if stream:
    print("🧠 Streaming thinking mode response:")
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            # Print the response chunk
            chunk_json = json.loads(chunk.get("bytes").decode())
            content_block_delta = chunk_json.get("choices")[0]["delta"].get("content")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = datetime.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")

                chunk_count += 1
                print(content_block_delta, end="")
    print(f"\nTotal chunks: {chunk_count}")
else:
    print("No response stream received.")


### Option 3: Amazon Bedrock's Converse API

The Bedrock Converse API provides a consistent interface for working with all Bedrock models that support messages. This means you can write your code once and use it across different models without changes. 

Key Benefits:
- Universal Interface: Same API structure works with Claude, Llama, Titan, and other models
- Model-Specific Parameters: Pass unique parameters when needed for specific models
- Privacy: Amazon Bedrock doesn't store any content you provide - data is only used for response generation
- Advanced Features: Built-in support for guardrails, tools/function calling, and prompt management
- **Thinking Mode Support**: Direct control over DeepSeek-V3.1's thinking capabilities

Additionally, the Converse API automatically separates the reasoning trace from the final response, giving developers the flexibility to show or hide the model's thinking process from end users based on their application needs.

In [ ]:
# Converse API with thinking mode
response = bedrock_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "How far from earth is the moon?"}]
        }
    ],
    system=[{"text": "You are a concise, highly logical assistant."}],
    inferenceConfig={
        "temperature": 0,
        "maxTokens": 1000
    },
    additionalModelRequestFields={
        "reasoning_effort": "high"  # Enable thinking mode for DeepSeek-V3.1
    }
)

# Message dict
print(f"📝 Message dict:")
print(response['output']['message']['content'])

# Reasoning trace (if available)
if 'reasoningContent' in response['output']['message']['content'][0]:
    print(f"📝 Reasoning trace:")
    print(response['output']['message']['content'][0]['reasoningContent']['reasoningText']['text'])

# Final response
print(f"📝 Final response:")
print(response['output']['message']['content'][1]['text'])


#### Streaming with Converse API

The Converse API comes with built in streaming support. This can be useful in user-facing applications since it reduces time to first token (TTFT) metric and with that perceived inference latency for the end user. The output below will contain reasoning trace and final response. Based on your application you might want to hide the reasoning trace from the end user.

In [ ]:
def bedrock_model_converse_stream(client, system_prompt, user_prompt, max_tokens=1000, temperature=0, reasoning_effort='high'):
    response = ""
    response = client.converse_stream(
        modelId=MODEL_ID,
        messages=[  
            {
                "role": "user",
                "content": [
                    {
                        "text": user_prompt
                    }
                ]
            },                        
        ],
        system=[{"text": system_prompt}],
        inferenceConfig={
            "temperature": temperature,
            "maxTokens": max_tokens
        },
        additionalModelRequestFields={
            "reasoning_effort": reasoning_effort  # Enable thinking mode for DeepSeek-V3.1
        }
    )
    # Extract and print the response text in real-time.
    for event in response['stream']:
        if 'contentBlockDelta' in event:
            chunk = event['contentBlockDelta']
            if chunk['delta'].get('reasoningContent', None):
                print(chunk['delta']['reasoningContent']['text'], end="")
            if chunk['delta'].get('text', None):
                print(chunk['delta']['text'], end="")
    return


In [ ]:
# Define input prompts
system_prompt = "You are a concise, highly logical assistant."
user_prompt = "How far from earth is the moon?"

# Invoke model through Converse API with streaming and thinking mode
print("🧠 Streaming with thinking mode:")
bedrock_model_converse_stream(bedrock_client, system_prompt, user_prompt, reasoning_effort='high')


## Conclusion

You've successfully explored **three powerful ways** to interact with DeepSeek-V3.1 on Amazon Bedrock, including comprehensive tool use capabilities and thinking mode!

### What We've Accomplished

**1. OpenAI SDK Integration**
- Set up the familiar OpenAI SDK to work seamlessly with AWS Bedrock
- Leveraged existing OpenAI patterns while running on AWS infrastructure
- Demonstrated streaming responses with real-time token generation
- **Implemented enhanced function calling** using familiar OpenAI SDK patterns
- **Explored thinking vs non-thinking modes** for different use cases

**2. Direct InvokeModel API Access**
- Implemented low-level Bedrock InvokeModel API calls for maximum control
- Built custom functions for both streaming and non-streaming inference
- Measured performance metrics like time-to-first-token for streaming responses
- Gained fine-grained control over model parameters and request formatting
- **Built custom tool use handling** with manual request/response processing
- **Direct control over thinking mode** through request parameters

**3. Bedrock Converse API**
- Explored the unified Converse API that works across all Bedrock models
- Demonstrated consistent message-based interactions regardless of underlying model
- Leveraged built-in support for system prompts and inference configuration
- **Integrated tool calling** using Bedrock's native toolSpec format
- **Thinking mode integration** through additionalModelRequestFields

**4. Enhanced Capabilities**
- Implemented the **same weather function across all three APIs** for direct comparison
- Learned the different schema formats and response handling approaches
- Understood when to choose each API based on your specific needs
- **Explored DeepSeek-V3.1's unique thinking capabilities**
- **Demonstrated enhanced tool calling performance**

### Key Benefits Achieved

✅ **Flexibility**: Three different API approaches for different use cases  
✅ **Performance**: Streaming support for improved user experience  
✅ **Familiarity**: Use existing OpenAI SDK patterns with AWS infrastructure  
✅ **Control**: Direct API access when you need fine-grained customization  
✅ **Consistency**: Universal interface that works across all Bedrock models  
✅ **Privacy**: AWS Bedrock doesn't store your data - only used for inference  
✅ **Tool Integration**: Enhanced function calling capabilities across all three approaches
✅ **Practical Comparison**: Side-by-side examples using the same function
✅ **Thinking Mode**: Step-by-step reasoning capabilities for complex problems
✅ **Hybrid Architecture**: Mixture-of-Experts design for optimal performance

### What's Next?

You're now equipped with comprehensive knowledge to choose the right API approach for your specific use case. Whether you need:
- The **simplicity** of the OpenAI SDK
- The **control** of InvokeModel 
- The **consistency** of Converse API
- **Enhanced tool use capabilities** for external integrations
- **Thinking mode** for complex reasoning tasks
- **Non-thinking mode** for quick responses

You have all the tools and examples to build powerful AI applications with DeepSeek-V3.1's advanced capabilities on Amazon Bedrock!